# Première approche de l'IA

L'Intelligence Artificielle appliquée aux jeux se heure à deux difficultés :

- la largeur du jeu (le nombre de coups possible à tour de jeu)
- la profondeur du jeu (le nombre de tours de jeu)

Dans des jeux comme les danes, les échecs ou pire le go, la largeur et la profondeur sont très importantes. Le nombre de parties possibles est donc immense ($10^{170}$ configurations légales possibles au go).

![go](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Go_game.jpg/220px-Go_game.jpg)


Dans cette étude, on va modéliser un jeu de morpion (tic-tac-toe) dans lequel on cherche à tour de rôle à aligner trois pions sur un plateau carré à 9 cases.

![morpion](https://upload.wikimedia.org/wikipedia/commons/3/33/Tictactoe1.gif)

Pour simplifier la programmation, on remplace les pions «X» et les «O» par les chiffres 1 et des 2 (0 pour une case vide) et on modélise le plateau $3\times 3$ par un tableau à deux dimensions dont les lignes et colonnes sont numérotées de 0,1,2 comme Python a l'habitude de numéroter ses listes.

       °  ¹  ²
    ° [2, 2, 1]
    ¹ [1, 2, 0]
    ² [0, 1, 2]
    
Sur le plateau ci-dessus, le joueur 2 a gagné.

Pour lancer chaque script, utilisez le bouton `▶| Exécuter` après avoir cliqué sur une cellule de code.

In [ ]:
plateau = [[1,2,0],[1,2,0],[0,1,2]]
print(plateau)

# Afficher correctement le plateau

Problème, on le voit l'affichage n'est pas très lisible.
On construit donc une fonction `affiche` qui présente le plateau de jeu sous une forme plus agréable.

La syntaxe `print("blabla %s" var)` affiche «blabla» suivi du contenu (transformé en chaîne de caractères) de la variable `var`.

In [ ]:
def affiche(pl):
    print("   °  ¹  ²")
    print("° %s" %pl[0])
    print("¹ %s" %pl[1])
    print("² %s" %pl[2])
    print()

In [ ]:
plateau = [[1,2,0],[1,2,0],[0,1,2]]
affiche(plateau)

# Vérifier si quelqu'un a gagné

Pour déterminer si pour un plateau donné, un des deux joueurs a gagné, on construit une fonction qui renvoie une valeur selon les cas :
- 0 si personne n'a encore gangé
- 1 si le joueur 1 a aligné trois pions
- 2 si le joueur 2 a aligné trois pions

On peut sans doute faire plus court mais cette définition de la fonction `qui_gagne` a l'avantage d'être simple : on se contente de vérifier les 8 manières de gagner pour chaque joueur.

In [ ]:
def qui_gagne(pl):
    for j in 1,2:
        if pl[0][0]==pl[0][1]==pl[0][2]==j : return j
        if pl[1][0]==pl[1][1]==pl[1][2]==j : return j
        if pl[2][0]==pl[2][1]==pl[2][2]==j : return j
        if pl[0][0]==pl[1][0]==pl[2][0]==j : return j
        if pl[0][1]==pl[1][1]==pl[2][1]==j : return j
        if pl[0][2]==pl[1][2]==pl[2][2]==j : return j
        if pl[0][0]==pl[1][1]==pl[2][2]==j : return j
        if pl[0][2]==pl[1][1]==pl[2][0]==j : return j
    return 0

Vérifions avec deux exemples

In [ ]:
plateau = [[1,2,0],[1,2,0],[0,1,2]]
affiche(plateau)
qui_gagne(plateau)
# Renvoie 0 car personne n'a gagné

In [ ]:
plateau = [[1,2,0],[0,2,1],[0,2,1]]
affiche(plateau)
qui_gagne(plateau)
# Renvoie 2 car les pions du joueur 2 sont alignés

# Laisser jouer l'humain

On construit maintenant une fonction qui interroge le joueur humain pour lui permettre de placer un pion numéro `numero`.

In [ ]:
def humain_joue(pl,numero):
    rejoue = True
    while rejoue:
        l = int(input("ligne (0,1 ou 2): "))
        c = int(input("colonne (0,1 ou 2): "))
        if pl[l][c]==0 : # Vérifie si case libre
            rejoue = False
            pl[l][c]=numero

Essayons de jouer :

In [ ]:
plateau = [[0,0,0],[0,0,0],[0,0,0]]

affiche(plateau)
humain_joue(plateau,1)
affiche(plateau)
humain_joue(plateau,1)
affiche(plateau)

# Laisser jouer la machine

Une fonction qui permet à l'ordinateur de placer un pion `numero` avec l'algorithme le plus simple : le hasard.

La machine choisit une ligne et une colonne au hasard jusqu'à trouver une place libre.

In [ ]:
from random import *

def ordi_joue(pl,numero):
    rejoue = True
    while rejoue:
        l = randint(0,2)
        c = randint(0,2)
        if pl[l][c]==0 :
            pl[l][c]=numero
            rejoue = False

plateau = [[0,0,0],[0,0,0],[0,0,0]]
affiche(plateau)
ordi_joue(plateau,2)
affiche(plateau)
ordi_joue(plateau,2)
affiche(plateau)

# Une partie Humain vs Machine

Dans la partie suivante l'humain joue les pions 1 et la machine les pions 2.

In [ ]:
plateau = [[0,0,0],[0,0,0],[0,0,0]]
affiche(plateau)
while True:
    humain_joue(plateau,1)
    affiche(plateau)
    if qui_gagne(plateau)==1 : break
    ordi_joue(plateau,2)
    affiche(plateau)
    if qui_gagne(plateau)==2 : break
print("victoire de %s" %qui_gagne(plateau))

# Première IA : jouer pour gagner

Une première startégie simple, plutôt que jouer au hasard, à cahque tour, on va chercher parmi les cases vides, celles qui permettraient de gagner la partie en un coup. 

Il nous faut donc d'abord une fonction qui liste les coordonnées des cases libres.

In [ ]:
def list_cases_libres(pl):
    L = []
    for i in 0,1,2:
        for j in 0,1,2:
            if pl[i][j]==0: L = L+[[i,j]]
    return L

plateau = [[1,2,0],[0,0,1],[0,2,1]]
affiche(plateau)
list_cases_libres(plateau)

Maintenant, on construit une fonction qui va tester chaque case libre et lui attribuer 1000 ou 0 points selon qu'elle permet de gagner ou non et qui renvoie une liste des points correspondant à la liste des cases libres. 

In [ ]:
def valeur_coups_profondeur_1(pl):
    pl_test = pl[:]
    L = []
    for pos in list_cases_libres(pl):
        pl_test[pos[0]][pos[1]]=2
        if qui_gagne(pl_test)==2:
            L = L + [1000]
        else:
            L = L + [0]
        pl_test[pos[0]][pos[1]]=0
    return L

In [ ]:
affiche(plateau)
print(list_cases_libres(plateau))
valeur_coups_profondeur_1(plateau)
# on voit que seule [1,1] correspond à la valeur 1000. Les autres coups valent 0.

On peut maintenant construire une fonction capable de placer 
- soit un pion gagnant lorsque c'est possible
- soit le pion au centre si la case est libre
- soit un pion au hasard si le gain de la partie n'est pas possible immédiatement.

In [ ]:
def ordi_joue_prof1(pl,numero):
    vcp1 = valeur_coups_profondeur_1(plateau)
    if max(vcp1)>0 :
        best = vcp1.index(max(vcp1))
        Z = list_cases_libres(plateau)
        l,c = Z[best][0], Z[best][1]
        pl[l][c] = numero
    elif (pl[1][1]==0):
        pl[1][1] = numero
    else :
        ordi_joue(pl,numero)

# Deux algos s'affrontent : hasard vs profondeur=1

On va donc lancer une partie entre une machine jouant au hasard (fonction `ordi_joue`) contre une machine jouant avec l'algorithme de profondeur 1 (fonction `ordi_joue_prof1`)

In [ ]:
plateau = [[0,0,0],[0,0,0],[0,0,0]]
print(" Plateau de départ : ")
affiche(plateau)
print("--------")

while True:
    print("Machine hasard joue : ")
    ordi_joue(plateau,1)
    affiche(plateau)
    print("--------")
    if qui_gagne(plateau)==1 : break

    print(" ordi prof1 joue:")
    ordi_joue_prof1(plateau,2)
    affiche(plateau)
    print("--------")
    if qui_gagne(plateau)==2 : break


print("victoire de %s" %qui_gagne(plateau))

# Seconde IA : jouer pour gagner en prévoyant le coup de l'adversaire

On va maintenant programmer une startégie en deux coups. Il va falloir construire l'ensemble des coups possibles sur un en trois coups :

MACHine -> ADVersaire -> MACHine.

- **Profondeur 1** : on part d'un plateau `pl`, on le copie dans `pl1` dans lequel on teste toutes les coups possibles. 
- **Profondeur 2** : pour chaque coup, on copie `pl1` dans `pl2` dans lequel on teste  toutes les réponses possibles.
- **Profondeur 3** : pour chaque réponse, on copie `pl3` dans `pl3` dans lequel on teste toutes les contres possibles.

C'est donc une stratégie dans laquelle l'arbre des possibilités sur trois niveaux est complètement testé.


In [ ]:
def valeur_coups_profondeur_3(pl,n_Mach,n_Adv):
    # liste des positions de niv.1 étudiées
    Lpos = []
    # nb victoires pour chaque position de niv.1
    nb_vic = []
    nb = 0

    pl1 = pl[:]
    # tous les coups possibles, profondeur p=1  
    for pos1 in list_cases_libres(pl): 
        pl1[pos1[0]][pos1[1]]=num_Machine
        if qui_gagne(pl1)==num_Machine: 
            # si on gagne du premier coup : fini
            # on renvoie le coup gagnant
            # après avoir remis pl1 en place
            pl1[pos1[0]][pos1[1]]=0
            return [pos1],[1]
        
        # si pas gagné du 1er coup toutes les
        # réponses de l'adversaire, prof. p=2
        pl2 = pl1[:]
        for pos2 in  list_cases_libres(pl1):
            pl2[pos2[0]][pos2[1]]=num_Adversaire
            if qui_gagne(pl2)==num_Adversaire:
                # si adversaire gagne ici,
                # alors il faut jouer à sa place
                # après avoir remis pl1 et pl2 en place
                pl1[pos1[0]][pos1[1]]=0
                pl2[pos2[0]][pos2[1]]=0
                return [pos2],[-1]
            else:
                
                # si adversaire n'a pas gagné en
                # réponse, tous les contres à la
                # réponse de l'adversaire, prof. p=3
                pl3 = pl2[:]
                for pos3 in  list_cases_libres(pl2):
                    pl3[pos3[0]][pos3[1]]=num_Machine
                    if qui_gagne(pl1)==num_Machine:
                        nb = nb+1
                    # RàZ avant étude du contre p=3 suivt 
                    pl3[pos3[0]][pos3[1]]=0
                    
                # RàZ avant étude de réponse p=2 suivt  
                pl2[pos2[0]][pos2[1]]=0
                
        Lpos = Lpos + [pos1]
        nb_vic = nb_vic + [nb]
        nb = 0
        # RàZ avant étude du coup p=1 suivant
        pl1[pos1[0]][pos1[1]]=0
        
    return Lpos,nb_vic

Testons cet algorithme sur un exemple pour voir que est selon lui le meilleur coup à jouer :

In [ ]:
plateau = [[1,0,2],[0,0,1],[0,2,0]]
affiche(plateau)
Lpos,nb_vic = valeur_coups_profondeur_3(plateau,n_Mach=2,n_Adv=1)
print(Lpos)
print(nb_vic)

best = nb_vic.index(max(nb_vic))
print("meilleur coup : ")
print(Lpos[best])

On peut maintenant construire une fonction capable de placer un pion au meilleur endroit possible après étude de l'arbre de profondeur 3.

In [ ]:
def ordi_joue_prof3(pl,numero):
    Lpos,nb_vic = valeur_coups_profondeur_3(plateau,n_Mach=2,n_Adv=numero+1%2)
    if (pl[1][1]==0):
        pl[1][1] = numero
    else :
        best = nb_vic.index(max(nb_vic))
        l,c = Lpos[best]
        pl[l][c] = numero

# Deux algos s'affrontent : hasard vs profondeur=3

On va donc lancer une partie entre une machine jouant au hasard (fonction `ordi_joue`) contre une machine jouant avec l'algorithme de profondeur 1 (fonction `ordi_joue_prof3`)

In [ ]:
plateau = [[0,0,0],[0,0,0],[0,0,0]]
print(" Plateau de départ : ")
affiche(plateau)
print("--------")

while True:
    print("Machine hasard joue : ")
    ordi_joue(plateau,1)
    affiche(plateau)
    print("--------")
    if qui_gagne(plateau)==1 : break

    print(" ordi prof3 joue:")
    ordi_joue_prof3(plateau,2)
    affiche(plateau)
    print("--------")
    if qui_gagne(plateau)==2 : break


print("victoire de %s" %qui_gagne(plateau))